In [1]:
import keras
import tensorflow as tf
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.keras.models import Model
from keras.applications import imagenet_utils
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

In [2]:
mobile = tf.keras.applications.mobilenet.MobileNet()

17227776/17225924 [==============================] - 0s 0us/step


In [3]:
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [4]:
base_model=MobileNet(weights='imagenet' , input_shape=(224 , 224 , 3) ,include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

17227776/17225924 [==============================] - 0s 0us/step


In [5]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [ ]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

In [ ]:
for layer in model.layers[:87]:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
# for layer in model.layers[:20]:
#     layer.trainable=False
# for layer in model.layers[20:]:
#     layer.trainable=True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/content/drive/My Drive/dataset/train',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 383 images belonging to 2 classes.


In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=12)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/12
11/11 [==============================] - 124s 11s/step - loss: 0.4797 - accuracy: 0.8466
Epoch 2/12
11/11 [==============================] - 1s 100ms/step - loss: 0.0923 - accuracy: 0.9573
Epoch 3/12
11/11 [==============================] - 1s 98ms/step - loss: 0.0247 - accuracy: 0.9943
Epoch 4/12
11/11 [==============================] - 1s 104ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 5/12
11/11 [==============================] - 1s 98ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 6/12
11/11 [==============================] - 1s 110ms/step - loss: 5.3456e-04 - accuracy: 1.0000
Epoch 7/12
11/11 [==============================] - 1s 101ms/step - loss: 3.1704e-05 - accuracy: 1.0000
Epoch 8/12
11/11 [==============================] - 1s 100ms/step - loss: 2.3756e-05 - accuracy: 1.0000
Epoch 9/12
11/11 [==============================] - 1s 102ms/step - loss: 1.0825e-05 - accuracy: 1.0000
Epoch 10/1

In [ ]:
validation_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

validation_generator=validation_datagen.flow_from_directory('/content/drive/My Drive/dataset/validation',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=False)

Found 96 images belonging to 2 classes.


In [ ]:
pred = model.evaluate(validation_generator , steps=12)
model.metrics_names
pred

24/24 [==============================] - 3s 115ms/step - loss: 0.0437 - accuracy: 0.9896


[0.04370713233947754, 0.9895833134651184]

In [ ]:
# save the model to tflite format
keras_file = "AI_Tunidex"
tf.saved_model.save(model, keras_file)

In [ ]:
# # load the file
# converter = tf.lite.TFLiteConverter.from_saved_model(keras_file)
# tflite_model = converter.convert()
# open("AI_Tunidex.tflite", "wb").write(tflite_model)